# Sistema criptográfico RSA

Ricardo Silva a71532

Francisca Fernandes a72450

João Cerqueira a65432

# Introdução

Com o crescimento da internet e a necessidade de transferencia de dados com segurança, começaram a ser desenvolvidos algoritmos criptográficos para assegurar a integridade dos dados, assim assegurando uma comunicação segura entre as duas partes, geralmente a estrutura cliente/servidor. O algoritmo RSA foi descrito em 1977 por Ron Rivest, Adi Shamir, e Leonard Adleman.

Este tipo de algoritmo é chamado de criptografia assimétrica ou de chave pública, pois existem duas chaves que são usadas.

É uma ideia simples mas que vem revolucionar a criptografia. Quem quiser comunicar de forma segura, encripta a mensagem com a chave pública do receptor e só ele consegue desencriptá-la com a respectiva chave privada, que não precisa nem deve ser partilhada com ninguém. Para além de estabelecer comunicações seguras, os sistemas criptográficos de chave pública têm por objectivo autenticar através de uma assinatura digital a informação transmitida.

Apesar de terem sido Dime e Hellman a introduzir o conceito, não apresentaram nenhum exemplo. São Rivest, Shamir e Adleman que, em Fevereiro de 1978 [RSA78], apresentam o primeiro sistema criptográfico de chave pública: o sistema criptográfico RSA.

# Geração de chaves 

A chave pública é o par (n,e).

A chave privada é d.

Vamos supor que um cliente querer enviar uma mensagem a um servidor:

Sejam p,q dois numeros primos aleatorios:

In [2]:
p = random_prime(1000, lbound=150)
q = random_prime(1000, lbound=150)
p, q

(911, 263)

Seja n=pq, entao φ(n)=φ(pq)=(p−1)(q−1) pois p,q sao primos.

$e\in \mathbb{Z}_{\varphi(n)}^*$, ou seja, $e\in \mathbb{Z}_{\varphi(n)}$ com $(e, \varphi(n))=1$. Calcula $d = e^{-1}$ em $\mathbb{Z}_{\varphi(n)}$.
​	 


In [3]:
n= p*q
m =(p-1)*(q-1)
Zn=IntegerModRing(n)
Zm=IntegerModRing(m)

In [11]:
e = 19
gcd(e, m)


1

In [12]:
n

239593

Então o par(239593,19) seria a chave publica do servidor.

A sua chave privada vai ser calculada a partir do e. 

In [13]:
d = power_mod(e, -1, m)
d

87839

In [14]:
ChPub = (n,e)
ChPriv = d

Possivel algoritmo para fazer esta geração de chaves seria:

In [15]:
def RSA_Key(nbits = 512):
    p = random_prime(2^(nbits//2), 2^(nbits//2-1))
    q = random_prime(2^(nbits//2+1), 2^(nbits//2))
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    while gcd(e, m) != 1:
        e = randint(2, m-1)
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    PrivKey = d
    return PubKey, PrivKey

# Cifração

Seja x a mensagem que o cliente quer enviar então , essa mensgaem vai ser cifrada numa mensagem y antes de ser enviada, e depois sera decifrada pelo servidor.

In [21]:
x = 123
y = power_mod(x, e, n)
y

170139

Um possivel Algoritmo para a cifração :

In [0]:
def RSA_encrypt(x, chave_pub):
    n, e = chave_pub
    criptograma = power_mod(x, e, n)
    return criptograma

# Decifração
O servidor vai decifrar a mensagem y e voltar a obter a mensagem x original.

In [24]:
power_mod(y, d, n)

123

Um possivel Algoritmo para a decifração :

In [2]:
def RSA_decrypt(y, chave):
    n, _ = chave[0]
    d = chave[1]
    z = power_mod(y, d, n)
    return z

# Assinaturas Digitais

Pode-se ao enviar uma mensagem querer também assiná-la, tal como se assina um documento ou um cheque. O sistema RSA pode também ser usado para esse fim, sendo possível associar a uma mensagem uma assinatura digital que não é mais que um número inteiro que garante a origem da mensagem.

# Ataques ao RSA
Após o desenvovimento deste sistema foram apontados problemas relativamente a segurança do mesmo.

Suponhamos que a mensagem x não é prima relativa com o modulo RSA, ou seja n.

Seja (x,n)!=1 e n=pq

Entao p|x ou q|x

e (x,n)= p ou (x,n)=q

Caso fossemos um atacante bastaria calcular (x,n) para ficar com a fatorização de n e portanto φ(n).

Como e é publico , sabendo p e q teriamos φ(n)=(p-1)(q-1) 

e portanto:

       d = power_mod(e,-1,φ(n))

Como contormamos esta situção?

Sabemos que :
        
        n = pq

e entao :

       φ(n)=n(1-1/p)(1-1/q)


Entao sabemos que o nº de numeros não primos relativos com n é 

        n-n(1-1/p)(1-1/q)
        
A probabilidade de x não ser primo realtivo com n é dada por 

        (n-n(1-1/p)(1-1/q))/n  = 1/p + 1/q - 1/pq

Entao sabemos que a probabilidade de uma mensagem x nao ser primo relativo com o modulo RSA , que nos leva a problemas de segurança é dada por :

    1/p + 1/q - 1/pq , com p e q fatores do modulo RSA.





Mas se o modulo do RSA foi sufecientemente grande sabemos que esta probabilidade vai ser muito baixa.

Atualmente chaves RSA tem um comprimento de 2048 bits , corresponde a que n tenha a volta de 600 digitos e p,q a volta de 300 digitos

Se o módulo N tem n-bits então os primos p e g devem ter n/2 bits cada, i.é, ter o mesmo tamanho (mas com os primeiros bits diferentes, por exemplo), pois quanto maior for um factor primo, mais difícil é encontrá-lo.

Temos tambem de ter algum cuidado na escolha dos expoentes, privado ou publico porque isso tambem nos pode levar a tipos de ataques que vamos descrever a frente de forma sussinta. 

# Expoente Privado Pequeno
Numa tentativa de tornar o processo de desencriptação mais rápido podemos cair na tentação de escolher um expoente privado d "pequeno". O teorema de M. Wiener mostra que a escolha de d "pequeno" pode levar à ruptura total da segurança do sistema criptográfico RSA, ou seja, consegue-se determinar o expoente privado d e consequentemente a factorização de N.

# Expoente Público Pequeno

No processo de encriptação podemos querer escolher um expoente público pequeno na tentativa de tornar a encriptação mais rápida.

Os ataques ao expoente público estão longe de levar à ruptura total do sistema RSA pois não chegam a factorizar o módulo N. O objectivo destes ataques é normalmente desencriptar uma mensagem sem precisar de descobrir a chave privada. 